# Artificial Neural Network

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.9.1'

## Part 1 - Data Processing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [4]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
dataset.isnull().any()

RowNumber          False
CustomerId         False
Surname            False
CreditScore        False
Geography          False
Gender             False
Age                False
Tenure             False
Balance            False
NumOfProducts      False
HasCrCard          False
IsActiveMember     False
EstimatedSalary    False
Exited             False
dtype: bool

In [7]:
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [8]:
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [9]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

### Encoding categorical data

Label Encoding the 'Gender' column

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [11]:
x

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

One Hot Encoding the 'Georgaphy' column

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [13]:
x

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

### Spliting the dataset into the Training set and Testing set

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [16]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [18]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [19]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#  updates weights^

### Training the ANN on the Training set

In [21]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5314 - accuracy: 0.7625
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4626 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4449 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4360 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4281 - accuracy: 0.7952
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4197 - accuracy: 0.8066
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4117 - accuracy: 0.8183
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4047 - accuracy: 0.8214
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3999 - accuracy: 0.8235
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3955 - accura

250/250 [==============================] - 0s 2ms/step - loss: 0.3393 - accuracy: 0.8615
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3389 - accuracy: 0.8614
Epoch 83/100
250/250 [==============================] - 0s 744us/step - loss: 0.3388 - accuracy: 0.8621
Epoch 84/100
250/250 [==============================] - 0s 672us/step - loss: 0.3385 - accuracy: 0.8605
Epoch 85/100
250/250 [==============================] - 0s 743us/step - loss: 0.3387 - accuracy: 0.8604
Epoch 86/100
250/250 [==============================] - 0s 700us/step - loss: 0.3382 - accuracy: 0.8612
Epoch 87/100
250/250 [==============================] - 0s 680us/step - loss: 0.3379 - accuracy: 0.8590
Epoch 88/100
250/250 [==============================] - 0s 704us/step - loss: 0.3382 - accuracy: 0.8627
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3380 - accuracy: 0.8627
Epoch 90/100
250/250 [==============================] - 0s 922us/step - loss: 0.337

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [25]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))>0.5)

1/1 [==============================] - 0s 14ms/step
[[0.05268255]]
1/1 [==============================] - 0s 14ms/step
[[False]]


### Predicting the Test set results

In [29]:
y_pred = ann.predict(x_test)

63/63 [==============================] - 0s 1ms/step


In [30]:
y_pred

array([[0.38839543],
       [0.23652789],
       [0.19997612],
       ...,
       [0.16479081],
       [0.14571542],
       [0.20272914]], dtype=float32)

In [33]:
y_pred = (y_pred > 0.5)
np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1)

array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int64)

### Making the Confusion Matrix

In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f'Accuracy Score = {round(accuracy_score(y_test, y_pred)*100,2)}%')

[[1516   79]
 [ 211  194]]
Accuracy Score = 85.5%
